# 1: Import thư viện và thiết lập đường dẫn

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import yaml
from datetime import datetime
from sqlalchemy import text

# Thêm thư mục chứa code vào path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Import Class mới
from connector import SQLServerConnector

# 2: Khởi tạo kết nối Database

In [ ]:
# 1. Load cấu hình từ file YAML
config_path = os.path.join('..', 'config', 'db_config.yaml')

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)['db_info'] # Giả sử cấu trúc yaml là db_info: {server: ..., ...}

# 2. Khởi tạo kết nối
# Lưu ý: Class mới sử dụng SQLAlchemy 2.0
db = SQLServerConnector(
    server=config['server'],
    database=config['database'],
    username=config['username'],
    password=config['password']
)

print(f"Đã khởi tạo engine kết nối tới: {db.server}/{db.database}")

# 3: Tạo dữ liệu giả lập (Mock Data)

In [ ]:
data = {
    'id_date': [20230101, 20230102, 20230103], 
    'product_name': ['Laptop Dell', 'Chuột Logitech', 'Bàn phím Cơ'], # Test tiếng Việt
    'quantity': [10, 50, 30],
    'price': ['15.5M', '200K', '500K'], 
    'created_at': [datetime.now(), datetime.now(), datetime.now()]
}
df = pd.DataFrame(data)
print("Dữ liệu mẫu (Pandas DataFrame):")
display(df)

# 4: Demo ```upsert_data``` - Tạo bảng và Insert lần đầu

In [ ]:
table_name = "Demo_Sales_Table_V2"

# (Optional) Xóa bảng cũ để chạy lại demo từ đầu
# V2 dùng SQLAlchemy engine để thực thi lệnh raw SQL
try:
    with db.get_engine().connect() as conn:
        conn.execute(text(f"DROP TABLE IF EXISTS {table_name}"))
        conn.commit()
    print(f"Đã xóa bảng {table_name} (nếu có) để demo lại.")
except Exception as e:
    print(f"Lỗi khi xóa bảng: {e}")

# --- THỰC HIỆN UPSERT ---
print(f"Đang đẩy dữ liệu vào bảng {table_name}...")

# primary_key='id_date': Cột này dùng để định danh duy nhất.
db.upsert_data(
    df=df, 
    target_table=table_name, 
    primary_key='id_date'
)

# Kiểm tra kết quả
df_result = db.get_data(table_name)
display(df_result)

**Cơ chế của hàm upsert_data trong V2:**

Thay vì kiểm tra từng dòng bằng Python (chậm), V2 sử dụng kỹ thuật **Staging Table + SQL MERGE**:

1. Upload dữ liệu vào bảng tạm (```##staging_...```).

2. Chạy lệnh SQL ```MERGE``` để so sánh bảng tạm và bảng chính dựa trên primary_key.

3. **WHEN MATCHED**: Cập nhật dữ liệu cũ (Update).

4. **WHEN NOT MATCHED**: Thêm mới (Insert).

**Ưu điểm:** Tốc độ cực nhanh và đảm bảo tính toàn vẹn dữ liệu.

### 4.1 Demo Upsert - Cập nhật dữ liệu và Thêm mới

In [ ]:
# Tạo Batch 2:
# - ID 20230102 (Chuột): Thay đổi quantity 50 -> 100 (Update)
# - ID 20230104 (Màn hình): Mới tinh (Insert)
data_batch_2 = {
    'id_date': [20230102, 20230104], 
    'product_name': ['Chuột Logitech (Updated)', 'Màn hình LG'],
    'quantity': [100, 5],          
    'price': ['200K', '3.5M'],
    'created_at': [datetime.now(), datetime.now()]
}
df_batch_2 = pd.DataFrame(data_batch_2)

print("Dữ liệu Batch 2 (Hỗn hợp Update và Insert):")
display(df_batch_2)

print(f"\n--- Bắt đầu Upsert Batch 2 ---")
db.upsert_data(
    df=df_batch_2, 
    target_table=table_name, 
    primary_key='id_date'
)

print(f"\n--- Kết quả trong DB (Chú ý dòng 20230102 đã đổi tên và số lượng) ---")
display(db.get_data(table_name))

In [ ]:
# Batch 3: Có thêm cột 'category' chưa từng có trong DB
data_batch_3 = {
    'id_date': [20230105],
    'product_name': ['Tai nghe Sony'],
    'quantity': [15],
    'price': ['2.5M'],
    'created_at': [datetime.now()],
    'category': ['Audio'] # <--- Cột mới
}
df_batch_3 = pd.DataFrame(data_batch_3)

print("--- Upsert dữ liệu có cột mới ---")
db.upsert_data(
    df=df_batch_3,
    target_table=table_name,
    primary_key='id_date',
    auto_evolve_schema=True # Mặc định là True, bật để tự thêm cột
)

print("Kết quả: Cột 'category' đã được thêm vào, các dòng cũ là NULL")
display(db.get_data(table_name))

# 5: Demo ```get_data``` nâng cao (Thay thế ```getData_slice```)

In [ ]:
print("1. Lấy toàn bộ bảng:")
# Truyền tên bảng
display(db.get_data(table_name))

print("\n2. Lấy dữ liệu với điều kiện SQL (Thay cho getData_slice):")
# Truyền câu lệnh SQL
# Ví dụ: Lấy Top 2 sản phẩm có quantity > 10
sql_query = f"""
    SELECT TOP 2 * FROM {table_name} 
    WHERE quantity > 10 
    ORDER BY id_date DESC
"""
df_custom = db.get_data(sql_query)
display(df_custom)

# 6: Đóng kết nối

In [ ]:
# Luôn dispose engine khi hoàn tất để giải phóng connection pool
db.dispose()
print("Đã đóng kết nối.")